In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import helper_functions_elliot as hfe # Elliot's helper function
from scipy import stats

wind = pd.read_csv('data/field01_fast.csv')

# Variable Creation and Formating

u_mean = np.mean(wind['u'])
v_mean = np.mean(wind['v'])
w_mean = np.mean(wind['w'])
temp_mean = np.mean(wind['temp'])

wind['u_turb'] = wind['u'] - u_mean
wind['v_turb'] = wind['v'] - v_mean
wind['w_turb'] = wind['w'] - w_mean
wind['temp_turb'] = wind['temp'] - temp_mean

wind['u_mom_flux'] = wind['u_turb'] * wind['w_turb']
wind['v_mom_flux'] = wind['v_turb'] * wind['w_turb']
wind['heat_flux'] = wind['w_turb'] * wind['temp_turb']
wind['TKE'] = 1/2 * (wind['u_turb']**2 + wind['v_turb']**2 + wind['w_turb']**2)

wind.head()

np.mean(wind['u_mom_flux']), np.mean(wind['v_mom_flux'])

bins = np.arange(1.5,4.5,.1)
plt.hist(wind['u'],bins=bins,edgecolor='black')

bins = np.arange(-1.5,4.5,.1)
plt.hist(wind['v'],bins=bins,edgecolor='black')

wind.head()

plt.plot(wind['time'],wind['temp'])
plt.xlabel('time (s)')
plt.ylabel('temp (K)')
plt.grid()
plt.show()
# indicates tempurature cooling down

plt.plot(wind['time'],wind['u'])
plt.xlabel('time (s)')
plt.ylabel('u velocity (m/s)')
plt.grid()
plt.show()

wind.head()

# Interpolation

def log_law(z,u_star,z0,L):
    k = 0.35 # von Karman constant
    return (u_star/k) * (np.log(z/z0) + 4.7*z / L)

u_mean_mom_flux = np.mean(wind['u_turb'] * wind['w_turb'])
v_mean_mom_flux = np.mean(wind['v_turb'] * wind['w_turb'])
mean_heat_flux = np.mean(wind['heat_flux'])

mean_heat_flux

k = .35 # von Karman constant
g = 9.8 # gravitational constant
z0 = .05 # roughness length
u_star = (u_mean_mom_flux**2 + v_mean_mom_flux**2)**.25 # friction velocity
L = (u_star**3 * temp_mean) / (k*g*w_mean_heat_flux)
z = np.arange(.5,50,.5)
u = log_law(z,u_star,z0,L)

mean_heat_flux

# Richardson's Number

# The data we have was taken at a 2m height above a homogenous surface.
# Estimated neutral or stable conditions

### Gradient Richardson Number

def gradient_richardson(wind): 
    g = 9.8; z = 2; k = 0.35 # gravitational constant, sensor height, von karman constant
    u_mean_mom_flux = np.mean(wind['u_turb'] * wind['w_turb'])
    v_mean_mom_flux = np.mean(wind['v_turb'] * wind['w_turb'])
    mean_heat_flux = np.mean(wind['heat_flux'])
    temp_mean = np.mean(wind['temp'])
    u_star = (u_mean_mom_flux**2 + v_mean_mom_flux**2)**.25 # friction velocity
    L = - (u_star**3 * temp_mean) / (k*g*mean_heat_flux) # Obukhov length
    theta_sl = - mean_heat_flux / u_star # surface level temp
    theta_gradient = theta_sl * (.74 + 4.7*z/L) / (k*z) # temp shear
    vel_gradient = (u_star/(k*z) + 4.7*u_star/(L*k))**2 # Businger-Dyer relationship for wind shear (du/dz^2 + dv/dz^)
    return ((g/temp_mean) * theta_gradient) / (vel_gradient)
gradient_richardson(wind)

### Flux Richardson Number

def flux_richardson(wind):
    g = 9.8; z = 2; k = 0.35 # gravitational constant, sensor height, von karman constant
    theta = .0853 # angular distance between basis vector and mean vector
    res = np.array([0,0])
    for i in range(len(wind)):
        vector = np.array([wind['u'][i], wind['v'][i]])
        vector_transform = rotate(vector,theta)
        res = np.vstack((res, vector_transform))
        
    df = pd.DataFrame(res); df = df.drop(0)
    u_turb = df.iloc[:,0] - np.mean(df.iloc[:,0])
    v_turb = df.iloc[:,1] - np.mean(df.iloc[:,1])
    u_mean_mom_flux = np.mean(u_turb * wind['w_turb'])
    v_mean_mom_flux = np.mean(v_turb * wind['w_turb'])
    mean_heat_flux = np.mean(wind['heat_flux'])
    temp_mean = np.mean(wind['temp'])
    
    u_star = (u_mean_mom_flux**2 + v_mean_mom_flux**2)**.25 # friction velocity
    L = - (u_star**3 * temp_mean) / (k*g*mean_heat_flux) # Obukhov length
    u_gradient = (1 + 4.7*z / L) * u_star / (k*z) # businger dyer approximation
    
    return ((g/temp_mean) * mean_heat_flux) / (u_mean_mom_flux*u_gradient)
flux_richardson(wind)

mean_vector = np.mean(wind['u']), np.mean(wind['v'])

def rotate(vector, theta):
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    rotation_matrix = np.array([[cos_theta, sin_theta],
                                [-sin_theta, cos_theta]])
    return np.dot(rotation_matrix, vector)

theta = .0853 # angular distance between basis vector and mean vector
res = np.array([0,0])
for i in range(len(wind)):
    vector = np.array([wind['u'][i], wind['v'][i]])
    vector_transform = rotate(vector,theta)
    res = np.vstack((res, vector_transform))
df = pd.DataFrame(res); df = df.drop(0)
u_turb = df.iloc[:,0] - np.mean(df.iloc[:,0])
v_turb = df.iloc[:,1] - np.mean(df.iloc[:,1])
u_mean_mom_flux = np.mean(u_turb * wind['w_turb'])
v_mean_mom_flux = np.mean(v_turb * wind['w_turb'])

u_mean_mom_flux, v_mean_mom_flux